In [ ]:
%matplotlib inline

#import sys
#sys.path.insert(0,'../PETITE/')
#import os.path
#from AllProcesses import *

import matplotlib
from matplotlib.font_manager import FontProperties
matplotlib.rcParams['text.latex.preamble'] = r'\usepackage{amsmath} \usepackage{amssymb}'
from matplotlib import rc
rc('text', usetex=True)

from tqdm import tqdm

from PETITE import ryan_shower 

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import FixedLocator

font0 = FontProperties()
font = font0.copy()
font.set_size(24)
font.set_family('serif')
labelfont=font0.copy()
labelfont.set_size(20)
labelfont.set_weight('bold')
legfontBig=font0.copy()
legfontBig.set_size(18)
legfontBig.set_weight('bold')

def set_size(w,h, ax=None):
    """ w, h: width, height in inches """
    if not ax: ax=plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w)/(r-l)
    figh = float(h)/(t-b)
    ax.figure.set_size_inches(figw, figh)

In [11]:
pre_made_shower = np.load("./RyanOutputs/AllParticles_GeVShower_Lead.npy", allow_pickle=True)

In [8]:
LDet0, RDet0 = 574.0, 2.5
def RFront(partV, LDet):
    RF = []
    for part in partV:
        x, y, z = part.get_r0()
        E, px, py, pz = part.get_p0()
        RF.append(np.sqrt((x + px/pz*(LDet-z))**2 + (y + py/pz*(LDet-z))**2))
    return np.array(RF)

In [10]:
SMPhotonShower = pre_made_shower[[ps.get_IDs()[0] == 22 for ps in PionShower]]
SMPhotonShowerPass = pre_made_shower[RFront(SMPhotonShower, LDet0) < RDet0]

AttributeError: 'numpy.ndarray' object has no attribute 'get_IDs'

In [ ]:
print(np.sum([pds.get_IDs()[-1] for pds in SMPhotonShowerPass])/np.sum([pds.get_IDs()[-1] for pds in SMPhotonShower]), np.sum([pds.get_IDs()[-1] for pds in PionDarkBremPass])/np.sum([pds.get_IDs()[-1] for pds in PionDarkBrem]))

In [ ]:
PhotonsPerPOT = 2.9 * 2.0 * 0.98823
RWFactor = PhotonsPerPOT/NSamp

In [ ]:
def FF(Eg, mV):
    xT = mV/(1.0773*Eg + 0.013716)
    expterm = 1154.0*np.exp(-24.42*xT**0.3174)
    if expterm > 1.0:
        expterm = 1.0
    return expterm

In [ ]:
b0 = np.logspace(-2, 2, 41)
SME = [[p0.get_p0()[0], p0.get_IDs()[-1]] for p0 in SMPhotonShower]
SMPE = [[p0.get_p0()[0], p0.get_IDs()[-1]] for p0 in SMPhotonShowerPass]

BSME = [[p0.get_p0()[0], p0.get_IDs()[-1]] for p0 in PionDarkBrem]
BSMPE = [[p0.get_p0()[0], p0.get_IDs()[-1]] for p0 in PionDarkBremPass]

plt.hist(np.transpose(SME)[0], bins=b0, weights=RWFactor*np.transpose(SME)[1], histtype='step', lw=3, color='C0', ls='-', label=r"$\mathrm{SM}$")
plt.hist(np.transpose(SMPE)[0], bins=b0, weights=RWFactor*np.transpose(SMPE)[1], histtype='step', lw=3, color='C0', ls='--', label=r"$\mathrm{SM,\ forward{-}going}$")
plt.hist(np.transpose(BSME)[0], bins=b0, weights=RWFactor*np.transpose(BSME)[1], histtype='step', lw=3, color='C1', ls='-', label=r"$m_V = 10\ \mathrm{MeV}$")
plt.hist(np.transpose(BSMPE)[0], bins=b0, weights=RWFactor*np.transpose(BSMPE)[1], histtype='step', lw=3, color='C1', ls='--', label=r"$\mathrm{BSM,\ forward{-}going}$")

plt.gca().legend(loc=1, fontsize=16)
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')
plt.gca().set_xlabel(r"$E_\gamma\ [\mathrm{GeV}]$", fontsize=24)
plt.gca().set_ylabel(r"$\mathrm{photons/bin/POT}$", fontsize=24)
#plt.gca().axis([1e-3,100,1e-11,1e-3])
[l.set_size(16) for l in plt.gca().get_xticklabels()]
[l.set_size(16) for l in plt.gca().get_yticklabels()]

set_size(12.0, 5.0)


In [ ]:
b0 = np.logspace(-2, 2, 41)
SMR = [[sm[0], FF(sm[0], 0.010)*sm[1]] for sm in SMPE]

plt.hist(np.transpose(SMPE)[0], bins=b0, weights=RWFactor*np.transpose(SMPE)[1], histtype='step', lw=3, color='#999999', ls='-', label=r"$\mathrm{SM,\ (3.38)}$")
plt.hist(np.transpose(SMR)[0], bins=b0, weights=RWFactor*np.transpose(SMR)[1], histtype='step', lw=3, color='C6', ls='-', label=r"$\mathrm{BSM,\ reweighting,\ (0.93)}$")
plt.hist(np.transpose(BSMPE)[0], bins=b0, weights=RWFactor*np.transpose(BSMPE)[1], histtype='step', lw=3, color='C4', ls='-', label=r"$\mathrm{BSM,\ resampling,\ (3.5\times 10^{-3})}$")

plt.gca().legend(loc=1, fontsize=16)
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')
plt.gca().set_xlabel(r"$E_\gamma\ [\mathrm{GeV}]$", fontsize=24)
plt.gca().set_ylabel(r"$\mathrm{photons/bin/POT}$", fontsize=24)
#plt.gca().axis([1e-3,100,1e-11,1e-3])
[l.set_size(16) for l in plt.gca().get_xticklabels()]
[l.set_size(16) for l in plt.gca().get_yticklabels()]

set_size(12.0, 5.0)


In [ ]:
print(np.sum(RWFactor*np.transpose(SMPE)[1]), np.sum(RWFactor*np.transpose(SMR)[1]), np.sum(RWFactor*np.transpose(BSMPE)[1]))

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display

In [ ]:
BSM3D = [[p0.get_p0()[0], np.sqrt(p0.get_p0()[1]**2 + p0.get_p0()[2]**2)/p0.get_p0()[3], p0.get_IDs()[-1]] for p0 in PionDarkBrem]
SM3D = [[p0.get_p0()[0], np.sqrt(p0.get_p0()[1]**2 + p0.get_p0()[2]**2)/p0.get_p0()[3], p0.get_IDs()[-1]] for p0 in SMPhotonShower]
SM3DRW = [[p0.get_p0()[0], np.sqrt(p0.get_p0()[1]**2 + p0.get_p0()[2]**2)/p0.get_p0()[3], FF(p0.get_p0()[0], 0.010)*p0.get_IDs()[-1]] for p0 in SMPhotonShower]


In [ ]:
figwid = 6.0
fighei = 6.0
lside = 3.0
rside = 3.5
wwspace = 0.5

ncol = 3
nrow = 1

wid = lside + ncol*figwid + (ncol-1)*wwspace + rside

bot = 3.77
top = 3.5
hhspace = 0.5

hei = bot + nrow*fighei + (nrow-1)*hhspace + top

lfactor = lside/wid
rfactor = rside/wid
bfactor = bot/hei
tfactor = top/hei
wfactor = wwspace/figwid
hfactor = hhspace/fighei

matplotlib.rcParams['axes.linewidth'] = 2.0
fig, axes = plt.subplots(nrow, ncol, figsize=(wid, hei), facecolor='1.0');
fig.subplots_adjust(left = lfactor, bottom=bfactor, right=(1.0-rfactor), top=(1.0-tfactor), wspace=wfactor, hspace=hfactor);

ax = axes[0]
ax.hist2d(np.transpose(SM3D)[0], np.transpose(SM3D)[1], bins=[np.logspace(-2, 2,41), np.logspace(-4,-1,31)], weights=RWFactor*np.transpose(SM3D)[2], norm=matplotlib.colors.LogNorm())

[l.set_size(16) for l in ax.get_xticklabels()]
[l.set_size(16) for l in ax.get_yticklabels()]

#plt.gca().legend(loc=2, fontsize=16)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylabel(r"$p_T/p_Z$", fontsize=24)
ax.set_xlabel(r"$E_\gamma\ [\mathrm{GeV}]$", fontsize=24)

ax = axes[1]
ax.hist2d(np.transpose(SM3DRW)[0], np.transpose(SM3DRW)[1], bins=[np.logspace(-2, 2,41), np.logspace(-4,-1,31)], weights=RWFactor*np.transpose(SM3DRW)[2], norm=matplotlib.colors.LogNorm())

[l.set_size(16) for l in ax.get_xticklabels()]
[l.set_size(-1) for l in ax.get_yticklabels()]

#plt.gca().legend(loc=2, fontsize=16)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylabel(r"", fontsize=24)
ax.set_xlabel(r"$E_V\ [\mathrm{GeV}]$", fontsize=24)

ax = axes[2]
ax.hist2d(np.transpose(BSM3D)[0], np.transpose(BSM3D)[1], bins=[np.logspace(-2, 2,41), np.logspace(-4,-1,31)], weights=RWFactor*np.transpose(BSM3D)[2], norm=matplotlib.colors.LogNorm())

[l.set_size(16) for l in ax.get_xticklabels()]
[l.set_size(-1) for l in ax.get_yticklabels()]

#plt.gca().legend(loc=2, fontsize=16)

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylabel(r"", fontsize=24)
ax.set_xlabel(r"$E_V\ [\mathrm{GeV}]$", fontsize=24)

In [ ]:
figwid = 6.0*16.0/9.0
fighei = 6.0
lside = 3.0
rside = 3.5
wwspace = 0.5

ncol = 1
nrow = 1

wid = lside + ncol*figwid + (ncol-1)*wwspace + rside

bot = 3.77
top = 3.5
hhspace = 0.5

hei = bot + nrow*fighei + (nrow-1)*hhspace + top

lfactor = lside/wid
rfactor = rside/wid
bfactor = bot/hei
tfactor = top/hei
wfactor = wwspace/figwid
hfactor = hhspace/fighei

matplotlib.rcParams['axes.linewidth'] = 2.0
fig, axes = plt.subplots(nrow, ncol, figsize=(wid, hei), facecolor='1.0');
fig.subplots_adjust(left = lfactor, bottom=bfactor, right=(1.0-rfactor), top=(1.0-tfactor), wspace=wfactor, hspace=hfactor);



ax = axes
#ax.hist(np.transpose(SM3D)[1], bins=np.logspace(-5,5,101), weights=RWFactor*np.transpose(SM3D)[2], histtype='step', lw=3, color='#999999')
ax.hist(np.transpose(SM3DRW)[1], bins=np.logspace(-5,5,26), weights=RWFactor*np.transpose(SM3DRW)[2], histtype='step', lw=3, color='C6', label=r"$\mathrm{Reweighted\ SM\ Photons}$")
ax.hist(np.transpose(BSM3D)[1], bins=np.logspace(-5,5,26), weights=np.sum(RWFactor*np.transpose(SM3DRW)[2])/np.sum(RWFactor*np.transpose(BSM3D)[2])*RWFactor*np.transpose(BSM3D)[2], histtype='step', lw=3, color='C4', label=r"$\mathrm{Resampled\ SM\ Shower}$")

[l.set_size(16) for l in ax.get_xticklabels()]
[l.set_size(16) for l in ax.get_yticklabels()]

ax.legend(loc='best', fontsize=16, framealpha=1.0)
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r"$p_T/p_Z$", fontsize=24)
ax.set_ylabel(r"$\mathrm{photons/bin\ (arb.\ norm.)}$", fontsize=24)
